## DESCARGA DE DATOS METEOROLÓGICOS Y DE CLIMA


In [1]:
# Instalando las paqueterías necesarias.
%pip install pandas openpyxl requests beautifulsoup4
%pip install wget
%pip install gdown
%pip install unidecode


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Las siguientes librerías nos ayudaran para el desarrollo del proceso de descarga y generación del archivo txt. es necesario que este al principio para su correcto funcionamiento.

Se utiliza librerías especiales como gwdown para descarga de archivos grandes, eliminando la confirmación de permiso de descarga.

Se utiliza librería unicode, para poder quitar los acentos de ciertas palabras y usarlo como nombre de archivos a descargar.

In [2]:
# Importación de librerías para usarlas en el proyecto de descarga y generación del txt con la descripción de esta
import pandas as pd
import wget
import gdown
import requests
import re
import os
import time
from bs4 import BeautifulSoup
from datetime import datetime
from logging import exception
from unidecode import unidecode
from datetime import datetime, timedelta

La inicialización de ciertas variables ayudaran a preparar la información inicial de cada una de ellas, además de que puedan ser usadas en cualquier parte del código o proceso, sin en el que perjudique el ámbito o espacio donde estas estén siendo seteadas.

In [3]:
# Inicializando variables que serán usadas más adelante

incidencia_clima_link = 'https://open-meteo.com/en/docs/historical-weather-api'
incidencia_title = ''
incidencia_description = ''

folder_path_data = '../rawdata'
folder_path_log = '../logs/descarga/'
folder_path_error_log = '../logs/errores/'
csv_filepath = ''

# Creando variable que contendra el listado de cada source descargado.
sources_list = []

# Creando variable que contendra el listado de cada nombre de archivo descargado y que esta siendo enviado a nuestro repositorio
names_files_list = []

texto_list = ''

#### Sección donde se encuentran las funciones generales del proceso

In [4]:
def log_error_write(description, line):
    # Verificando si la carpeta existe, si no, crearla
    if not os.path.exists(folder_path_error_log):
        os.makedirs(folder_path_error_log)

    date_now = datetime.now()

    # Variable que contiene el texto que tendrá el archivo txt que se generará y descargará.
    line_error = f"""{date_now.strftime('%Y-%m-%d %H:%M:%S')} {line} - {description}"""
    # Se crea el archivo a partir del texto de arriba, el cual se guardará con el nombre descriptivo más la fecha día*mes*año, para identificar la descarga por día.
    with open(f'{folder_path_error_log}/ddf_err{date_now.strftime("%d%m%y")}.txt', 'a', encoding="utf-8") as error_file:
        error_file.write(f"\n{line_error}")

def log_write(description, line):
    # Verificando si la carpeta existe, si no, crearla
    if not os.path.exists(folder_path_log):
        os.makedirs(folder_path_log)

    date_now = datetime.now()

    # Variable que contiene el texto que tendrá el archivo txt que se generará y descargará.
    line_write = f"""{date_now.strftime('%Y-%m-%d %H:%M:%S')} {line} - {description}"""
    # Se crea el archivo a partir del texto de arriba, el cual se guardará con el nombre descriptivo más la fecha día*mes*año, para identificar la descarga por día.
    with open(f'{folder_path_log}/muncoord{date_now.strftime("%d%m%y")}.txt', 'a', encoding="utf-8") as write_file:
        write_file.write(f"\n{line_write}")


def create_csv_from_DataFrame(df, csv_filename):
    try:

        # Construyendo la ruta completa del archivo
        csv_filepath = os.path.join(folder_path_data, csv_filename)

        # Guardando el DataFrame como un archivo CSV
        df.to_csv(csv_filepath, index=False, encoding='utf-8')
        print(f"DataFrame guardado exitosamente en {csv_filepath}")

    except Exception as e:
        # Imprimiendo un mensaje de error en caso de excepción
        log_error_write(f"Error al guardar el DataFrame como CSV: {e}", 'create_csv_from_DataFrame')
        print(f"Error al guardar el DataFrame como CSV: {e}")

#### Web Scraping para obtener las coordenadas geográficas de todos los municipios de Sonora

In [5]:
# Inicializando listas para almacenar los datos
ciudades = []
latitudes = []
longitudes = []

try:

  for num in range(2):
    url = f'https://www.coordenadas.com.es/mexico/pueblos-de-sonora/26/{num + 1}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    pretty_soup = soup.prettify()

  # Encontrando el elemento tabla desde el html obtenido
    tabla = soup.find('table', class_='table')

    # Iterarando sobre las filas de la tabla
    for fila in tabla.find_all('tr'):
        # buscando y encontrando las celdas de la fila
        celdas = fila.find_all('td')

        # Verificando si hay celdas (evitando encabezados de la tabla)
        if celdas:
            # Extrayendo el texto de la etiqueta <a> dentro de la primera celda
            ciudad = celdas[0].find('a').text.strip()
            # Extrayendo el texto de la segunda celda
            coordenada = celdas[1].text.strip()

            # Dividiendo las coordenadas en latitude y longitude
            latitud, longitud = map(float, coordenada.split(','))

            ciudades.append(ciudad)
            latitudes.append(latitud)
            longitudes.append(longitud)

  coordenadas_df = pd.DataFrame({
     'municipio': ciudades,
     'latitude': latitudes,
     'longitude': longitudes
     })


  #llenando la informacion faltante

  Mmunicipios_left_df = pd.DataFrame({
     'municipio': ["Magdalena", "Mazatán", "Moctezuma", "Rayón", "Rosario", "Santa Ana", "Villa Hidalgo", "Benito Juárez"],
     'latitude': [30.733333, 28.933333, 29.806111, 29.710556, 28.05, 30.543889, 30.25, 27.1],
     'longitude': [-111.05, -110.166667, -109.679444, -110.570278, -109.3, -111.121111, -109.333333,  -109.85]
  })

  coordenadas_df = pd.concat([coordenadas_df,Mmunicipios_left_df], ignore_index = True)

  create_csv_from_DataFrame(coordenadas_df, f'municipios_coordenadas.csv')

  # Mostrar el DataFrame resultante
  print(coordenadas_df)

except ValueError as ve:
  # Manejo de errores para entradas no válidas (por ejemplo, si no se ingresa un número)
  log_error_write(f"Error de valor: {ve}", 'web scraping coordenadas')
  print(f"Error de valor: {ve}")

except Exception as e:
  # Manejo de errores para cualquier otro tipo de excepción no anticipada
  log_error_write(f"Ocurrió un error inesperado: {e}", 'web scraping coordenadas')
  print(f"Ocurrió un error inesperado: {e}")

DataFrame guardado exitosamente en ../rawdata\municipios_coordenadas.csv
        municipio   latitude   longitude
0         Aconchi  29.825040 -110.227170
1     Agua-Prieta  30.948490 -109.274370
2          Alamos  27.026940 -108.936590
3           Altar  30.713610 -111.835280
4        Arivechi  28.927990 -109.188300
..            ...        ...         ...
67          Rayón  29.710556 -110.570278
68        Rosario  28.050000 -109.300000
69      Santa Ana  30.543889 -111.121111
70  Villa Hidalgo  30.250000 -109.333333
71  Benito Juárez  27.100000 -109.850000

[72 rows x 3 columns]


#### Descarga de datos históricos climatológicos y meteorológicos  

In [6]:
# Calculando la fecha anterior restando un día
date_yesterday = datetime.now() - timedelta(days=1)
end_date_str = date_yesterday.strftime('%Y-%m-%d')

api_url = "https://archive-api.open-meteo.com/v1/archive"

wheater_df =  pd.DataFrame()
coordenadas_ignore = pd.DataFrame()

try:

    for index,row in coordenadas_df.iterrows():

        print(row["municipio"])

        parameters = {
            "latitude": row['latitude'],
            "longitude": row['longitude'],
            "start_date": "2015-01-01",
            "end_date": end_date_str,
            "daily": "temperature_2m_max,temperature_2m_min,temperature_2m_mean,sunrise,sunset,precipitation_sum,rain_sum,precipitation_hours",
            "timezone": "auto"
            }

        # Realizando la solicitud GET a la API con los parámetros
        response = requests.get(api_url, params=parameters)

        if response.status_code == 429:
             coordenadas_ignore = pd.concat([coordenadas_ignore, row.to_frame().transpose()], ignore_index=True)
             print(f"{row['municipio']} - Demasiadas solicitudes. Esperando....")
             time.sleep(5) # Espera 5 segundos antes de intentar nuevamente
             continue

        # Verificando si la solicitud fue exitosa (código 200)
        if response.status_code == 200:
            # Convirtiendo la respuesta JSON en un DataFrame de pandas
            download_wheater_df = pd.DataFrame(response.json())

            if download_wheater_df is not None:
              # Obteniendo las claves (nombres de las series) de clima_df['daily']
              daily_series = download_wheater_df['daily'].keys()

              # Creando un nuevo DataFrame dinámico
              wheater_municipio_df = pd.DataFrame({clave: download_wheater_df['daily'][clave] for clave in daily_series})

              wheater_municipio_df['municipio'] = row['municipio']

              wheater_df = pd.concat([wheater_df,wheater_municipio_df], ignore_index = True)

              log_write(row['municipio'],f"Descarga-coordenada")

        time.sleep(120) # Espera 120 segundos entre solicitudes para no superar la tasa limite de descarga

        # Verificar cada 10 iteraciones
        if (index + 1) % 10 == 0:
             time.sleep(50)

    create_csv_from_DataFrame(wheater_df, f'DatosClimaMunicipal.csv')
    create_csv_from_DataFrame(coordenadas_ignore, f'DatosClimaMunicipal_ignorados')

except Exception as e:
        # Imprimir un mensaje de error en caso de excepción
        log_error_write(f"Ocurrió un error inesperado: {e}", 'decarga_datos_wheater')
        print(f"Error: {e}")

Aconchi
Agua-Prieta
Alamos
Altar
Arivechi
Arizpe
Atil
Bacadehuachi
Bacanora
Bacerac
Bacoachi
Bacum
Banamichi
Baviacora
Bavispe
Benjamin-Hill
Caborca
Cajeme
Cananea
Carbo
Cucurpe
Cumpas
Divisaderos
Empalme
Etchojoa
Fronteras
General-Plutarco-Elias-Calles
Granados
Guaymas
Hermosillo
Huachinera
Huasabas
Huatabampo
Huepac
Imuris
La-Colorada
Naco
Nacori-Chico
Nacori-Chico - Demasiadas solicitudes. Esperando....
Nacozari-de-Garcia
Nacozari-de-Garcia - Demasiadas solicitudes. Esperando....
Navojoa
Navojoa - Demasiadas solicitudes. Esperando....
Nogales
Nogales - Demasiadas solicitudes. Esperando....
Onavas
Onavas - Demasiadas solicitudes. Esperando....
Opodepe
Opodepe - Demasiadas solicitudes. Esperando....
Oquitoa
Oquitoa - Demasiadas solicitudes. Esperando....
Pitiquito
Pitiquito - Demasiadas solicitudes. Esperando....
Puerto-Penasco
Quiriego
Sahuaripa
San-Felipe-de-Jesus
San-Ignacio-Rio-Muerto
San-Javier
San-Luis-Rio-Colorado
San-Miguel-de-Horcasitas
San-Pedro-de-La-Cueva
Santa-Cruz
Saric


#### Descarga de datos históricos climatológicos y meteorológicos

> (version test, con 5 llamaddos solamente para no saturar y que provoque error 429)

In [6]:
# Calculando la fecha anterior restando un día
date_yesterday = datetime.now() - timedelta(days=1)
end_date_str = date_yesterday.strftime('%Y-%m-%d')

api_url = "https://archive-api.open-meteo.com/v1/archive"

wheater_df =  pd.DataFrame()
coordenadas_ignore = pd.DataFrame()

try:

    for index,row in coordenadas_df.iterrows():
        if index < 5:
            print(row["municipio"])

            parameters = {
                "latitude": row['latitude'],
                "longitude": row['longitude'],
                "start_date": "2015-01-01",
                "end_date": end_date_str,
                "daily": "temperature_2m_max,temperature_2m_min,temperature_2m_mean,sunrise,sunset,precipitation_sum,rain_sum,precipitation_hours",
                "timezone": "auto"
                }

            # Realizando la solicitud GET a la API con los parámetros
            response = requests.get(api_url, params=parameters)

            if response.status_code == 429:
                coordenadas_ignore = pd.concat([coordenadas_ignore, row.to_frame().transpose()], ignore_index=True)
                print(f"{row['municipio']} - Demasiadas solicitudes. Esperando....")
                time.sleep(5) # Espera 5 segundos antes de intentar nuevamente
                continue

            # Verificando si la solicitud fue exitosa (código 200)
            if response.status_code == 200:
                # Convirtiendo la respuesta JSON en un DataFrame de pandas
                download_wheater_df = pd.DataFrame(response.json())

                if download_wheater_df is not None:
                    # Obteniendo las claves (nombres de las series) de clima_df['daily']
                    daily_series = download_wheater_df['daily'].keys()

                    # Creando un nuevo DataFrame dinámico
                    wheater_municipio_df = pd.DataFrame({clave: download_wheater_df['daily'][clave] for clave in daily_series})

                    wheater_municipio_df['municipio'] = row['municipio']

                    wheater_df = pd.concat([wheater_df,wheater_municipio_df], ignore_index = True)

                    log_write(row['municipio'],f"Descarga-coordenada")

            time.sleep(20) # Espera 20 segundos entre solicitudes para no superar la tasa limite de descarga

            # Verificar cada 10 iteraciones
            if (index + 1) % 10 == 0:
                time.sleep(50)

    create_csv_from_DataFrame(wheater_df, f'DatosClimaMunicipal.csv')
    create_csv_from_DataFrame(coordenadas_ignore, f'DatosClimaMunicipal_ignorados.csv')

except Exception as e:
        # Imprimir un mensaje de error en caso de excepción
        log_error_write(f"Ocurrió un error inesperado: {e}", 'decarga_datos_wheater')
        print(f"Error: {e}")

Aconchi
Agua-Prieta
Alamos
Altar
Arivechi
DataFrame guardado exitosamente en ../rawdata\DatosClimaMunicipal.csv
DataFrame guardado exitosamente en ../rawdata\DatosClimaMunicipal_ignorados.csv


#### Sección que crea el archivo txt de la descripción de las fuentes descargadas

In [8]:
# Creando el archivo de texto con descripción general de las fuentes de descargar y parte del proceso ETL

try:

  # Verificando si la carpeta existe, si no, crearla
  if not os.path.exists(folder_path_log):
    os.makedirs(folder_path_log)

  # Uniendo elementos de la lista de las fuentes en donde se han descargado los datos (sources_list)
  texto_list = "\n".join(sources_list)

  date_now = datetime.now()

  # Variable que contiene el texto que tendrá el archivo txt que se generará y descargará.
  description = f"""
  Proyecto:
  Narrando una historia con datos: explorando el poder de la ciencia de datos

  Análisis de la variabilidad del comportamiento delictivo
  en relación con factores temporales y geográficos, y su correlación con el desarrollo social y urbano para la prevención del delito

  Equipo participante:
  - Miguel Ernesto Medina León
  - Luis Andrés Burruel Durán
  - Mario Estrada Ferreira

  Este cuaderno de trabajo está destinado a la adquisición de raw data de los datos climatológicos a nivel municipal. Se busca obtener una tabla de datos específicas:

  1. Datos clima municipal.

  Descripción de los datos descargados:
  - Los datos utilizados en este proyecto fueron adquiridos a través de la API Historical Weather. Se encuentra disponible información adicional
    acerca de esta API en el siguiente sitio web: https://open-meteo.com/en/docs/historical-weather-api

  
  Fuentes:
  https://archive-api.open-meteo.com/v1/archive

  Diccionario de datos:

    * Datos clima municipal

      - time: fecha correspondiente al día en el que se realizaron las mediciones climáticas.
      - temperature_2m_max: temperatura máxima diaria registrada a una altura de dos metros sobre el nivel del suelo.
      - temperature_2m_min: temperatura mínima diaria registrada a una altura de dos metros sobre el nivel del suelo.
      - sunrise: hora diaria en la cual el Sol se eleva sobre el horizonte.
      - sunset: hora diaria en la cual el Sol se oculta bajo el horizonte.
      - rain_sum: cantidad acumulada de precipitación diaria.
      - precipitation_hours: número de horas con precipitación registradas en un día específico.
      - temperature_2m_mean: temperatura promedio diaria registrada a dos metros sobre el nivel del suelo.
      - precipitacion_sum: suma total de la precipitación diaria, incluyendo lluvia, nieve, llovizna, entre otras formas de precipitación.
      - municipio: municipio en el cual se llevaron a cabo las mediciones climáticas correspondientes.


Coordenadas geográficas de los municipios de Sonora

Estas coordenadas geográficas representan la ubicación geoespacial de los municipios dentro del estado de Sonora y han sido recuperadas de la fuente: https://www.coordenadas.com.es/mexico/pueblos-de-sonora/26/
    

  Fechas de descarga:
  - El proceso de descarga de los datos se realizó el: {date_now.strftime('%Y-%m-%d %H:%M:%S')}
  """
  # Se crea el archivo a partir del texto de arriba, el cual se guardará con el nombre descriptivo más la fecha día*mes*año, para identificar la descarga por día.
  with open(f'{folder_path_log}/fuentes_descripcion{date_now.strftime("%d%m%y")}_Clima.txt', 'w', encoding='utf-8') as description_file:
      description_file.write(description)

  print("Archivos de datos descargados y descripción en txt creada.")

except ValueError as ve:
  # Manejo de errores para entradas no válidas (por ejemplo, si no se ingresa un número)
  log_error_write(f"Error de valor: {ve}", 'Creación de archivo de descripción general')
  print(f"Error de valor: {ve}")

except Exception as e:
  # Manejo de errores para cualquier otro tipo de excepción no anticipada
  log_error_write(f"Ocurrió un error inesperado: {e}", 'Creación de archivo de descripción general')
  print(f"Ocurrió un error inesperado: {e}")

Archivos de datos descargados y descripción en txt creada.
